In [1]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch
from transformers import pipeline
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

2023-05-20 12:04:03.930492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 12:04:04.283387: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-20 12:04:04.283421: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-20 12:04:05.490018: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')

In [3]:
modelqa = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

In [4]:
classifier = pipeline("zero-shot-classification",
                      model="joeddav/xlm-roberta-large-xnli")

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def findAction(sentence,candidate_labels = ["Take", "Drop", "Perform an action"]):
    result=classifier(sentence, candidate_labels)
    action=result['labels']
    score=result['scores']
    index=action.index("Perform an action")
    action.remove("Perform an action")
    score.pop(index)
    return action,score

In [6]:
result=findAction("Next to the crate there is a yellow cube that you must grab.")
print("result: ",result)

result:  (['Take', 'Drop'], [0.14090073108673096, 0.02239922806620598])


In [7]:
def findTarget(text,action):
    if action=="Take":
        question="What should be taken?"
    else:
        question="Where should we drop the object?"
    inputs = tokenizer(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = modelqa(**inputs)
    answer_start_index = torch.argmax(outputs.start_logits)
    answer_end_index = torch.argmax(outputs.end_logits)
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    return tokenizer.decode(predict_answer_tokens)

In [8]:
findTarget("Next to the crate there is a yellow cube that you must grab.","Take")

'a yellow cube'

Pour les réponses à des questions plus complexes le modèle DistillBert semble insuffisant:

In [26]:
def askQuestion(text,question):
    inputs = tokenizer(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = modelqa(**inputs)
    answer_start_index = torch.argmax(outputs.start_logits)
    answer_end_index = torch.argmax(outputs.end_logits)
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    return tokenizer.decode(predict_answer_tokens)

On voit qu'il répond mal même dans des cas très simples comme ci-dessous:

In [27]:
askQuestion('Context: the cube is taken. Order: "Take the cube. Then you must put it in the casse"', "What should I do now?")


'take the cube. then you must put it in the casse'

In [28]:
askQuestion('Context: the cube is taken. Order: "Take the cube, Then you must put it in the casse"', "What should I do now?")


'you must put it in the casse'

In [29]:
askQuestion('Each box must contain a cube and a sphere.', 'What object is the container?')

'a cube and a sphere'

Le modèle Roberta finetunner pour le SQAD semble plus adapté:

In [30]:
model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [31]:
QA_input = {
    'question': "What should I do now?",
    'context': 'Context: the cube is taken. Order: "Take the cube. Then you must put it in the casse"'
}
res = nlp(QA_input)
res['answer']

'put it in the casse'

In [32]:
def askQuestion(text,question):
    QA_input = {
        'question': question,
        'context': text
    }
    res = nlp(QA_input)
    return res['answer']

In [51]:
def askAction(state,order,question="What should I do now?"):
    QA_input = {
        'question': question,
        'context': 'Context: '+state+ 'Order: '+order
    }
    res = nlp(QA_input)
    return res['answer']

In [52]:
askQuestion('Context: the cube is taken. Order: "Take the cube, Then you must put it in the casse"', "What should I do now?")


'put it in the casse'